In [28]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as func

In [29]:
spark = SparkSession.builder.appName("Processing Articles").getOrCreate()

df = spark.read \
    .option("header", False) \
    .option("delimiter", "\t") \
    .option("inferSchema", True) \
    .csv("wiki.txt") \
    .toDF("url", "title", "text")

In [30]:
df.show(10)

+--------------------+--------------------+--------------------+
|                 url|               title|                text|
+--------------------+--------------------+--------------------+
|http://ru.wikiped...|               Литва|Литва  Литва́ (),...|
|http://ru.wikiped...|              Россия|Россия  Росси́я (...|
|http://ru.wikiped...|            Слоновые|Слоновые  Слоно́в...|
|http://ru.wikiped...|             Мамонты|Мамонты  Ма́монты...|
|http://ru.wikiped...|       Красная книга|Красная книга  Пе...|
|http://ru.wikiped...|           Соционика|Соционика  Социо́...|
|http://ru.wikiped...|               Школа|Школа  Первоначал...|
|http://ru.wikiped...|         Лингвистика|Лингвистика  Линг...|
|http://ru.wikiped...|          Социология|Социология  Социо...|
|http://ru.wikiped...|Древнерусское гос...|Древнерусское гос...|
+--------------------+--------------------+--------------------+
only showing top 10 rows



### 1. Напишите программу, которая находит самое длинное слово.

На неочищеном от ссылок датасете, самое длинное слово явзяется ссылкой

In [31]:
longest_word = df.select(func.split("text", "\s+").alias("words")) \
                 .selectExpr("explode(words) as word") \
                 .select(func.length("word").alias("word_length"), "word") \
                 .orderBy(func.desc("word_length")) \
                 .select("word") \
                 .first()[0]

print("Самое длинное слово: {}".format(longest_word))

Самое длинное слово: [https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%85%D0%B0%D0%BD%D0%B8%D0%BA%D0%B0_%D0%BA%D0%BE%D0%BD%D1%82%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B3%D0%BE_%D0%B2%D0%B7%D0%B0%D0%B8%D0%BC%D0%BE%D0%B4%D0%B5%D0%B9%D1%81%D1%82%D0%B2%D0%B8%D1%8F#.D0.AD.D0.BD.D0.B5.D1.80.D0.B3.D0.B8.D1.8F_.D0.BF.D1.80.D0.B8_.D1.83.D0.BF.D1.80.D1.83.D0.B3.D0.BE.D0.BC_.D0.BA.D0.BE.D0.BD.D1.82.D0.B0.D0.BA.D1.82.D0.B5]


Отчистим датасет от ссылок, получили самым длинным словом химический термин. Дальше буду работать с данными без ссылок

In [32]:
clean_text = df.withColumn("text", func.regexp_replace("text", r'http[s]?://\S+', ""))
clean_text = clean_text.withColumn("text", func.regexp_replace("text", r'http[s]?://\S+', ""))
words = clean_text.select(func.split("text", "\s+").alias("words")) \
                  .selectExpr("explode(words) as word")

longest_word = words.select(func.length("word").alias("word_length"), "word") \
                    .orderBy(func.desc("word_length")) \
                    .select("word") \
                    .first()[0]

print("Самое длинное слово: {}".format(longest_word))


Самое длинное слово: 10-[3-[4-(2-гидроксиэтил)пиперидин-1-ил]пропил]-N,N-диметилфенотиазин-2-сульфонамид.


### 2. Напишите программу, которая находит среднюю длину слов. 

In [33]:
avg_word_length = words.select(func.sum(func.length("word")).alias("total_length"), func.count("word").alias("count_words")) \
                       .select((func.col("total_length") / func.col("count_words")).alias("avg_length")).collect()[0][0]

print("Средняя длина слова:", avg_word_length)

Средняя длина слова: 6.53011518849989


### 3. Напишите программу, которая находит самое частоупотребляемое слово, состоящее из латинских букв.

In [34]:
latin_words = words.select(func.regexp_extract("word", "[a-zA-Z]+", 0).alias("latin_word")) \
                   .filter(func.col("latin_word") != "")

In [35]:
desc_frequency_words = latin_words.select(func.lower("latin_word").alias("latin_word")) \
                                  .groupBy("latin_word").count() \
                                  .orderBy(func.col("count").desc())

desc_frequency_words.show()

+----------+-----+
|latin_word|count|
+----------+-----+
|   formula|11587|
|         c| 4905|
|         i| 4753|
|        ii| 3723|
|       xix| 3040|
|        xx| 2896|
|       the| 2397|
|       iii| 2035|
|         x| 1828|
|         a| 1728|
|     xviii| 1717|
|         d| 1333|
|         v| 1326|
|      xvii| 1294|
|        iv| 1200|
|       xvi| 1172|
|         n| 1164|
|        of| 1120|
|         b| 1081|
|    codice| 1037|
+----------+-----+
only showing top 20 rows



In [36]:
most_common_word = desc_frequency_words.select("latin_word").first()[0]

print("Самое частое латинское слово:", most_common_word)

Самое частое латинское слово: formula


### 4. Все слова, которые более чем в половине случаев начинаются с большой буквы и встречаются больше 10 раз.

In [37]:
capitalize_words_count = words.filter(func.regexp_extract("word", r"^[А-ЯA-Z]", 0) != "") \
                              .groupBy("word").agg(func.count("*").alias("capitalize_words_count")) \
                              .orderBy((func.col("capitalize_words_count").desc()))

In [38]:
all_words_count = words.select(func.lower("word").alias("word")) \
                               .groupBy("word").agg(func.count("*").alias("all_words_count")) \
                               .orderBy(func.col("all_words_count").desc())

In [39]:
matching_words = all_words_count.join(capitalize_words_count,
                                      all_words_count["word"] == func.lower(capitalize_words_count["word"]),
                                      "inner") \
                                .select(
                                    all_words_count["word"],
                                    all_words_count["all_words_count"],
                                    capitalize_words_count["capitalize_words_count"])

In [40]:
#Создаем SQL-представление, также будем использовать в задании №7
matching_words.createOrReplaceTempView("words")

# Выполняем SQL-запрос
result = spark.sql('''
    SELECT *
    FROM words
    WHERE capitalize_words_count > all_words_count / 2
    ORDER BY capitalize_words_count DESC
''')

# Выводим результат
result.show()

+----------+---------------+----------------------+
|      word|all_words_count|capitalize_words_count|
+----------+---------------+----------------------+
|    россии|           5161|                  5159|
|    однако|           7804|                  4370|
|      ссср|           3809|                  3809|
|       сша|           3108|                  3108|
|российской|           3200|                  2749|
|     кроме|           3937|                  2714|
|        xx|           2659|                  2659|
|         i|           2593|                  2505|
|       xix|           2489|                  2489|
|  согласно|           3983|                  2310|
|        ii|           2243|                  2242|
|      так,|           2188|                  1619|
|     таким|           2957|                  1586|
|        а.|           1643|                  1570|
|    совета|           2099|                  1491|
|   россии.|           1429|                  1429|
|   россии,|

### 5. Напишите программу, которая с помощью статистики определяет устойчивые сокращения вида `пр.`, `др.`, ...

In [41]:
clean_words = words.withColumn("word", func.regexp_replace("word", "[^а-яА-Я0-9.\\s]", "")) \
                   .filter(func.col("word") != "")

In [42]:
abbr_words = clean_words.select(func.lower("word").alias("word")) \
                        .filter(func.regexp_extract("word", r"([^.][а-я]*[^.]\.$)", 1) != "") \
                        .select("word", (func.length('word').alias("length"))) \
                        .groupBy("word").count()

percentile = abbr_words.approxQuantile("count", [0.9995], 0)[0]
abbr_words = abbr_words.select("word").where((func.col("count") > percentile) & (func.length("word") <= 4))

abbr_words.show()

+----+
|word|
+----+
| гг.|
|год.|
| ст.|
| км.|
| ул.|
| им.|
| св.|
| вв.|
| др.|
|сша.|
|руб.|
|век.|
|чел.|
| см.|
|тыс.|
|лет.|
+----+



### 7. Напишите программу, которая с помощью статистики находит имена, употребляющиеся в статьях. 

Используем SQL представление из 4-го задания, имена начинаются с большой буквы, потому меняем условие, при котором происходит отбор слов

In [43]:
result = spark.sql('''
    SELECT *
    FROM words
    WHERE capitalize_words_count == all_words_count
    ORDER BY capitalize_words_count DESC
''')

result.show()

+---------+---------------+----------------------+
|     word|all_words_count|capitalize_words_count|
+---------+---------------+----------------------+
|     ссср|           3809|                  3809|
|      сша|           3108|                  3108|
|       xx|           2659|                  2659|
|      xix|           2489|                  2489|
|  россии.|           1429|                  1429|
|  россии,|           1383|                  1383|
|    xviii|           1325|                  1325|
|александр|           1188|                  1188|
|  украины|           1160|                  1160|
|     сша,|            987|                   987|
|    ссср.|            962|                   962|
|     xvii|            955|                   955|
|   москве|            930|                   930|
|       рф|            874|                   874|
|     сша.|            862|                   862|
|  франции|            855|                   855|
|      оао|            848|    